In [1]:
import os
import re 
import urllib

import requests as rq

def multiple_replace(dic, text):
  # Create a regex from the dict keys
  regex = re.compile('(%s)' % '|'.join(map(re.escape, dic.keys())))

  # For each match, lookup corresponding value in dict
  return regex.sub(lambda mo: dic[mo.string[mo.start():mo.end()]], text) 

## Load latest version notebook CSS
+ Latest version is 5.1.0 - as of 21sep17 - from the jupyter notebook repo: [here](https://cdn.jupyter.org/notebook/5.1.0/style/style.min.css)
+ For some unknown reason nbconvert is stuck at notebook css version 4.3.0 - See [setup.py](https://github.com/jupyter/nbconvert/blob/8f8ac1a8cf9aa718e9a8e6c9bc986d7c052ef5d2/setup.py#L69) 

In [2]:
def load_remote_file(url, login='login', password='',
                     proxies=False, proxy_host='', proxy_port=''):

    pwd = urllib.parse.quote(password)

    if proxies:
        dic = {'login': login, 'pwd': pwd,
               'proxy_host': proxy_host, 'proxy_port': proxy_port}
        proxies = {
            'http': 'http://{login}:{pwd}@{proxy_host}:{proxy_port}'.format(**dic),
            'https': 'https://{login}:{pwd}@{proxy_host}:{proxy_port}'.format(**dic)
        }

    res = rq.get(url, proxies=proxies)
    assert res.status_code == 200, 'Download error'
    return res.content.decode('utf-8')

    
def write_file(string, path):
    with open(path, 'w') as f:
        f.write(string)
    print('file {} saved on disk'.format(path))


version = '5.1.0'
url = 'https://cdn.jupyter.org/notebook/{}/style/style.min.css'.format(version)
notebook_css = load_remote_file(url)

notebook_css[:100]

'/*!\n*\n* Twitter Bootstrap\n*\n*/\n/*!\n * Bootstrap v3.3.7 (http://getbootstrap.com)\n * Copyright 2011-2'

## Define custom CSS
+ To hide the `Out[]` prompts
+ To slightly reduce the left margin width - when the `In[]` and `Out[]` prompts go away

In [41]:
prompt_css = """
.output_prompt {
    visibility: hidden;
}
.prompt {
    min-width: 5%; //14px
}
"""

ribbon_css = """

/* The ribbons */

.corner-ribbon{
  width: 200px;
  background: #e43;
  position: relative;
  top: 25px;
  left: -50px;
  text-align: center;
  line-height: 50px;
  letter-spacing: 1px;
  color: #f0f0f0;
  transform: rotate(-45deg);
  -webkit-transform: rotate(-45deg);
}

/* Custom styles */

.corner-ribbon.sticky{
  position: fixed;
}

.corner-ribbon.shadow{
  box-shadow: 0 0 3px rgba(0,0,0,.3);
}

/* Different positions */

.corner-ribbon.top-left{
  top: 25px;
  left: -50px;
  transform: rotate(-45deg);
  -webkit-transform: rotate(-45deg);
}

.corner-ribbon.top-right{
  top: 25px;
  right: -50px;
  left: auto;
  transform: rotate(45deg);
  -webkit-transform: rotate(45deg);
}

.corner-ribbon.bottom-left{
  top: auto;
  bottom: 25px;
  left: -50px;
  transform: rotate(45deg);
  -webkit-transform: rotate(45deg);
}

.corner-ribbon.bottom-right{
  top: auto;
  right: -50px;
  bottom: 25px;
  left: auto;
  transform: rotate(-45deg);
  -webkit-transform: rotate(-45deg);
}

/* Colors */

.corner-ribbon.white{background: #f0f0f0; color: #555;}
.corner-ribbon.black{background: #333;}
.corner-ribbon.grey{background: #999;}
.corner-ribbon.blue{background: #39d;}
.corner-ribbon.green{background: #2c7;}
.corner-ribbon.turquoise{background: #1b9;}
.corner-ribbon.purple{background: #95b;}
.corner-ribbon.red{background: #e43;}
.corner-ribbon.orange{background: #e82;}
.corner-ribbon.yellow{background: #ec0;}
"""

## Define nbconvert jinja template
+ To inject the recent notebook CSS
+ To circle in RED cells (`In` and `Out`) marked as "# NBCONVERT IMPORTANT" (typically first line of code cell)
+ To remove cells (`In` and `Out`) marked as "# NBCONVERT REMOVE" (typically first line of code cell)


In [42]:
template = """
{% extends 'full.tpl'%}

{% block header %}
{{ super() }}
<style>
__notebook_css__
</style>
{% endblock header %}

{% block any_cell %}
{% if cell.get('source', '').startswith('# NBCONVERT IMPORTANT') %}
    <div style="border:thin solid red">
        {{ super() }}
    </div>
{% elif cell.get('source', '').startswith('# NBCONVERT REMOVE') %}
    <div>
    </div>
{% else %}
    {{ super() }}
{% endif %}
{% endblock any_cell %}
"""

dic = {'__notebook_css__': notebook_css}

template = multiple_replace(dic, template)

path = 'template_selective.tpl'
write_file(template, path)

file template_selective.tpl saved on disk


In [43]:
# template = """
# {% extends 'full.tpl'%}



# {% block any_cell %}
# {% if cell.get('source', '').startswith('# NBCONVERT IMPORTANT') %}
#     <div style="border:thin solid red">
#         {{ super() }}
#     </div>
# {% elif cell.get('source', '').startswith('# NBCONVERT REMOVE') %}
#     <div>
#     </div>
# {% else %}
#     {{ super() }}
# {% endif %}
# {% endblock any_cell %}
# """

# # dic = {'__notebook_css__': notebook_css}

# template = multiple_replace(dic, template)

# path = 'template_selective.tpl'
# write_file(template, path)

## Define nbconvert jinja template
+ To inject the recent notebook CSS
+ To remove all `In` cells and display only outputs
+ To hide the `Out[]` prompts and slightly reduce the left margin


In [50]:
template = """
{% extends 'full.tpl'%}

{% block header %}
{{ super() }}
<style>
__notebook_css__
__prompt_css__
__ribbon_css__
</style>
{% endblock header %}

{% block body %}
    <div class="corner-ribbon top-right sticky blue">IMPORTANT</div>
    {{ super() }}
{% endblock body %}


{% block input_group %}
{% endblock input_group %}

{% block any_cell %}
{% if cell.get('source', '').startswith('# NBCONVERT IMPORTANT') %}
    <div style="border:3px solid orange; background-color: #f7f7f7">
        {{ super() }}
    </div>
{% elif cell.get('source', '').startswith('# NBCONVERT REMOVE') %}
    <div>
    </div>
{% else %}
    {{ super() }}
{% endif %}
{% endblock any_cell %}
"""

dic = {'__notebook_css__': notebook_css,
       '__prompt_css__': prompt_css,
       '__ribbon_css__': ribbon_css
      }

template = multiple_replace(dic, template)

path = 'template_hide_input.tpl'
write_file(template, path)

file template_hide_input.tpl saved on disk


## Choose notebook and template
+ To run nbconvert


In [51]:
nb_filename = 'sample_notebook.ipynb'

# template = 'template_selective.tpl'
template = 'template_hide_input.tpl'

print('Run nbconvert with\n\tnotebook\t{}\n\ttemplate\t{}'.format(nb_filename, template))

Run nbconvert with
	notebook	sample_notebook.ipynb
	template	template_hide_input.tpl


In [52]:
!jupyter nbconvert --to html $nb_filename --template=$template

[NbConvertApp] Converting notebook sample_notebook.ipynb to html
[NbConvertApp] Writing 541441 bytes to sample_notebook.html


In [64]:
!conda list nbconvert

# packages in environment at /usr/local/anaconda3:
#
nbconvert                 5.3.1            py36h810822e_0  
